In [1]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [2]:
def lr_schedule(epoch):
    initial_lr = 0.001
    if epoch < 10:
        return initial_lr
    else:
        return initial_lr * 0.1

In [3]:
data = pd.read_csv("../data/diabetes.csv")
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [4]:
X = data.drop(["Outcome"], axis=1)
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [5]:
y = data["Outcome"]
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.35, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [7]:
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

optimizer = Adam(learning_rate=lr_schedule(0))
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
lr_scheduler = LearningRateScheduler(lr_schedule)
early_stopper = EarlyStopping(monitor='val_loss', patience=20)
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val), callbacks=[lr_scheduler, early_stopper])

Epoch 1/100
32/32 [==============================] - 2s 41ms/step - loss: 0.6694 - accuracy: 0.6238 - val_loss: 0.5345 - val_accuracy: 0.7164
Epoch 2/100
32/32 [==============================] - 0s 5ms/step - loss: 0.4910 - accuracy: 0.7623 - val_loss: 0.5086 - val_accuracy: 0.7313
Epoch 3/100
32/32 [==============================] - 0s 5ms/step - loss: 0.5029 - accuracy: 0.7601 - val_loss: 0.4839 - val_accuracy: 0.7537
Epoch 4/100
32/32 [==============================] - 0s 5ms/step - loss: 0.4416 - accuracy: 0.7869 - val_loss: 0.4908 - val_accuracy: 0.7463
Epoch 5/100
32/32 [==============================] - 0s 5ms/step - loss: 0.4383 - accuracy: 0.8053 - val_loss: 0.4923 - val_accuracy: 0.7612
Epoch 6/100
32/32 [==============================] - 0s 5ms/step - loss: 0.4280 - accuracy: 0.7746 - val_loss: 0.4865 - val_accuracy: 0.7537
Epoch 7/100
32/32 [==============================] - 0s 5ms/step - loss: 0.4256 - accuracy: 0.7921 - val_loss: 0.5067 - val_accuracy: 0.7612
Epoch 8/100


In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

5/5 [==============================] - 0s 3ms/step - loss: 0.5172 - accuracy: 0.7630
Test accuracy: 0.7629629373550415
